In [1]:
#configure the necessary Spark environment
import os, sys

#del os.environ['PYSPARK_SUBMIT_ARGS']

#here you should set where are located the spark files
spark_home = '/dados/app/spark-1.6.2-bin-hadoop2.6/'

os.environ["SPARK_HOME"]  = spark_home

sys.path.insert(0, spark_home + "/python")

sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.3-src.zip'))

#this defines the sparkContext sc
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.2
      /_/

Using Python version 2.7.12 (default, Jul  1 2016 15:12:24)
SparkContext available as sc, HiveContext available as sqlContext.


In [2]:
import sys, os, math, datetime, shutil
from timeit import default_timer as timer
import re, unicodedata
from nltk.tag import pos_tag 
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.stem import RSLPStemmer
from nltk.corpus import stopwords
from pymongo import MongoClient
from openpyxl import load_workbook

from pyspark import SparkContext

from pyspark.ml.feature import HashingTF, IDF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.classification import SVMWithSGD, SVMModel


from pyspark.sql import Row, SQLContext

from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel

def createMongoDBConnection(host, port, username, password, db):
    client = MongoClient(host, port)
    return client[db]

def removeAccents(s):
  s = ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))
  return re.sub(r'[^\w]', ' ', s)

def findProductsByCategory(categories):
    
    db = createMongoDBConnection(host, port, username, password, database)
    produtos = db.produto_novo
    
    product_list = []
    query_filter = {}
    if categories:
      query_filter = {"categorias" : {"$in" : categories}}
    
    for produto in produtos.find(query_filter):
        keys = produto.keys()
        description = ''
        if 'descricaoLonga' in keys:
            description = removeAccents(description + produto['descricaoLonga'])
        if 'nome' in keys:
            description = removeAccents(description + produto ['nome'])
        id = None
        if '_id' in keys:
            id = str(produto['_id'])

        category = ''
        subcategory = ''

        if 'categorias' in keys:
            categorias_0 = removeAccents(produto['categorias'][0])

            if categorias_0 == "Inicio":            
                category = removeAccents(produto['categorias'][1])
                if(len(produto['categorias']) > 2):
                    subcategory = removeAccents(produto['categorias'][2])
            else:
                category = categorias_0
                if(len(produto['categorias']) > 1):
                    subcategory = removeAccents(produto['categorias'][1])

        if len(description)>0:
            product_list.append((id, description, category, subcategory))
    
    return product_list
def insertTokensAndCategories(tokens, category, categoryAndSubcategory):
    db = createMongoDBConnection(host, port, username, password, database)

    modelCollection = db.model
    modelCollection.remove({'_type':'token'})

    document_mongo =  dict()
    document_mongo['_type'] = 'token'
    document_mongo['_datetime'] = datetime.datetime.utcnow()
    i = 0
    for t in tokens:
        document_mongo[t] = i
        i = i + 1   

    modelCollection.insert_one(document_mongo)

    modelCollection.remove({'_type':'category'})

    document_mongo =  dict()
    document_mongo['_type'] = 'category'
    document_mongo['_datetime'] = datetime.datetime.utcnow()
    i = 0
    for c in category:
        document_mongo[c] = i
        i = i + 1 

    modelCollection.insert_one(document_mongo)

    modelCollection.remove({'_type':'category and subcategory'})
    
    document_mongo =  dict()
    document_mongo['_type'] = 'category and subcategory'
    document_mongo['_datetime'] = datetime.datetime.utcnow()
    i = 0
    for c in categoryAndSubcategory:
        document_mongo[c[0]+","+c[1]] = i
        i = i + 1 

    modelCollection.insert_one(document_mongo)

In [3]:
host = 'localhost'
port = 27017
username = ''
password = ''
database = 'tcc-recsys-mongo'

APP_NAME = 'Recomender System - Treinamento dos Modelos'
sqlContext = SQLContext(sc)

stpwrds = stopwords.words('english')
tbl_translate = dict.fromkeys(i for i in xrange(sys.maxunicode) if unicodedata.category(unichr(i)).startswith('S') or unicodedata.category(unichr(i)).startswith('P') or unicodedata.category(unichr(i)).startswith('N'))

In [4]:
print '---Pegando produtos---'
start_i = timer()
productRDD = sc.parallelize(findProductsByCategory([]))
print '####levou %d segundos' % (timer()-start_i)

---Pegando produtos---
####levou 12 segundos


In [5]:
print '---Criando corpus---'
start_i = timer()
corpusRDD = (productRDD.map(lambda s: (s[0], word_tokenize(s[1].translate(tbl_translate).lower()), s[2], s[3]))
                       .map(lambda s: (s[0], [PorterStemmer().stem(x) for x in s[1] if x not in stpwrds], s[2], s[3] ))
                       .map(lambda s: (s[0], [x[0] for x in pos_tag(s[1]) if x[1] == 'NN' or x[1] == 'NNP'], s[2], s[3]))
                       .cache())
print '####levou %d segundos' % (timer()-start_i)


---Criando corpus---
####levou 0 segundos


In [6]:
def getTokensAndCategories():  
    db = createMongoDBConnection(host, port, username, password, database)
    model = db.model
    
    tokens_dict = db.model.find({"_type": "token"}).limit(1).next()
    del tokens_dict['_type']
    del tokens_dict['_id']
    del tokens_dict['_datetime']
    tokens_list = [None] * (max(tokens_dict.values()) + 1)

    for key, value in tokens_dict.iteritems():
        tokens_list[value] = key

    categories_dict = db.model.find({"_type": "category"}).limit(1).next()
    del categories_dict['_type']
    del categories_dict['_id']
    del categories_dict['_datetime']
    categories_list = [None] * (max(categories_dict.values()) + 1)

    for key, value in categories_dict.iteritems():
        categories_list[value] = key

    categories_and_subcategories_dict = db.model.find({"_type": "category and subcategory"}).limit(1).next()
    del categories_and_subcategories_dict['_type']
    del categories_and_subcategories_dict['_id']
    del categories_and_subcategories_dict['_datetime']
    categories_and_subcategories_list = [None] * (max(categories_and_subcategories_dict.values()) + 1)

    for key, value in categories_and_subcategories_dict.iteritems():
        pre_string = key.split(",")
        categories_and_subcategories_list[value] = (pre_string[0], pre_string[1])

    return tokens_list, categories_list, categories_and_subcategories_list

In [7]:
tokens, category, categoryAndSubcategory = getTokensAndCategories()

In [8]:
#print '---Pegando e persistindo dados de categoria e tokens---'
# o original que criava essas infos no mongo, porem agora que estou reexecutando, esta dando diferena no numero de tokens
#start_i = timer()http://localhost:8888/notebooks/Treinando%20os%20classificadores.ipynb#
#tokens = corpusRDD.flatMap(lambda x: x[1]).distinct().collect()
numTokens = len(tokens)
print numTokens
#category = productRDD.map(lambda x: x[2]).distinct().collect()
#categoryAndSubcategory = productRDD.map(lambda x: (x[2], x[3])).distinct().collect()
#insertTokensAndCategories(tokens, category, categoryAndSubcategory)
#print '####levou %d segundos' % (timer()-start_i)  

54239


In [9]:
print '---Calculando TF-IDF dos produtos---'
start_i = timer()
wordsData = corpusRDD.map(lambda s: Row(label=s[0], words=s[1], category=s[2], subcategory=s[3]))
#persistir isso para que ele nao tenha que fazer de novo na predicaoo
wordsDataDF = sqlContext.createDataFrame(wordsData)   

wordsDataForPrediction = corpusRDD.map(lambda s: Row(label=s[0], words=s[1], type=s[2], originalpost = ''))
#persistir isso para que ele nao tenha que fazer de novo na predicaoo
wordsDataForPredictionDF = sqlContext.createDataFrame(wordsDataForPrediction)   

if os.path.exists("/home/felipe/Documentos/TCC/Experimento/spark_cluster/spark-1.6.2-bin-hadoop2.6/wordsDataDF.parquet"):
    shutil.rmtree("/home/felipe/Documentos/TCC/Experimento/spark_cluster/spark-1.6.2-bin-hadoop2.6/wordsDataDF.parquet")

wordsDataForPredictionDF.write.parquet("/home/felipe/Documentos/TCC/Experimento/spark_cluster/spark-1.6.2-bin-hadoop2.6/wordsDataDF.parquet") 

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=numTokens)
idf = IDF(inputCol="rawFeatures", outputCol="features")

featurizedData = hashingTF.transform(wordsDataDF)
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

#VSM = rescaledData.map(lambda t: LabeledPoint(categoryAndSubcategory.index((t.category, t.subcategory)), t.features))
VSM = rescaledData.map(lambda t: LabeledPoint(category.index(t.category), t.features))

VSMTrain, VSMTest = VSM.randomSplit([8, 2], seed=0L)
print '####levou %d segundos' % (timer()-start_i)    

---Calculando TF-IDF dos produtos---
####levou 175 segundos


In [32]:
print '--Criando modelo Naive Bayes---'
start_i = timer()
model = NaiveBayes.train(VSMTrain)

if os.path.exists("/dados/models/naivebayes/modelo_categoria"):
    shutil.rmtree("/dados/models/naivebayes/modelo_categoria")

model.save(sc, '/dados/models/naivebayes/modelo_categoria')
print '####levou %d segundos' % (timer()-start_i)    

--Criando modelo Naive Bayes---
####levou 71 segundos


In [33]:
print '---Testando modelo Naive Bayes---'
start_i = timer()
prediction = VSMTest.map(lambda p : (categoryAndSubcategory[int(model.predict(p.features))], categoryAndSubcategory[int(p.label)]))
acuraccy = float(prediction.filter(lambda (x, v): x[0]==v[0]).count())/float(prediction.count())
print 'acuracidade de %f' % (acuraccy*100)
print '####levou %d segundos' % (timer()-start_i)   

---Testando modelo Naive Bayes---
acuracidade de 93.116783
####levou 29 segundos


In [34]:
print '---Pegando os posts---'
start_i = timer()
posts = list()
wb = load_workbook(filename = '/home/felipe/Documentos/TCC/Experimento/ml_module/base_sentimentos.xlsx')
sheet = wb['Menes']
for row in sheet.iter_rows(row_offset=1):
    post = list()
    for cell in row:
        if cell.value is None:
            break
        post.append(1 if cell.value == 'Positive' or cell.value == 'Neutral' else 0 if cell.value == 'Negative' else removeAccents(cell.value))

    if len(post) > 0:            
        posts.append(tuple(post))

print '####levou %d segundos' % (timer()-start_i)

---Pegando os posts---
####levou 55 segundos


In [35]:
print '---Criando corpus---'
start_i = timer()
postsRDD = sc.parallelize(posts)
postCorpusRDD = (postsRDD.map(lambda s: (s[1], word_tokenize(s[0].translate(tbl_translate).lower())))
                       .map(lambda s: (s[0], [PorterStemmer().stem(x) for x in s[1] if x not in stpwrds]))
                       .map(lambda s: (s[0], [x[0] for x in pos_tag(s[1]) if x[1] == 'NN' or x[1] == 'NNP']))
                       .cache())

print '####levou %d segundos' % (timer()-start_i)

---Criando corpus---
####levou 0 segundos


In [36]:
print '---Calculando TF-IDF dos Posts---'
start_i = timer()
wordsData = postCorpusRDD.map(lambda s: Row(label=s[0], words=s[1]))
wordsDataDF = sqlContext.createDataFrame(wordsData)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=numTokens)
idf = IDF(inputCol="rawFeatures", outputCol="features")

featurizedData = hashingTF.transform(wordsDataDF)
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)   

VSM = rescaledData.map(lambda t: LabeledPoint(t.label, t.features))
VSMTrain, VSMTest = VSM.randomSplit([8, 2], seed=0L)
print '####levou %d segundos' % (timer()-start_i)  

---Calculando TF-IDF dos Posts---
####levou 41 segundos


In [37]:
print '--Criando modelo SVM---'
start_i = timer()
model = SVMWithSGD.train(VSMTrain, iterations=100)

if os.path.exists("/dados/models/svm"):
    shutil.rmtree("/dados/models/svm")

model.save(sc, "/dados/models/svm")
print '####levou %d segundos' % (timer()-start_i)   

--Criando modelo SVM---
####levou 17 segundos


In [38]:
print '---Testando modelo SVM---'
start_i = timer()
prediction = VSMTest.map(lambda p: (int(p.label), model.predict(p.features)))
acuraccy = prediction.filter(lambda (v, p): v == p).count() / float(prediction.count())

print 'acuracidade de %f' % (acuraccy*100)
print '####levou %d segundos' % (timer()-start_i)   

---Testando modelo SVM---
acuracidade de 80.262046
####levou 5 segundos
